MiBig is a curated resource of characterized natural products and their source organisms. This notebook takes as input a csv file of organisms from the MiBig website, searches NCBI for genome data and retrieves it.

In [42]:
import numpy as np
import pandas as pd
import urllib2
import urllib
import gzip
from bs4 import BeautifulSoup
import time

In [2]:
mibig = pd.read_excel('mibig.xlsx')
mibig.head()

,MIBiG accession,Full / partial MIBiG info,Main product,Biosynthetic class,Organism
0,BGC0000001,Full,abyssomicin,Polyketide,Verrucosispora maris AB-18-032
1,BGC0000016,Full,amphotericin,Other,Streptomyces nodosus
2,BGC0000017,Full,anatoxin,Alkaloid / Polyketide,Oscillatoria sp. PCC 6506
3,BGC0000023,Full,aurafuron,Polyketide,Stigmatella aurantiaca DW4/3-1
4,BGC0000024,Full,aureothin,Polyketide,Streptomyces thioluteus


In [3]:
urlbase = "https://www.ncbi.nlm.nih.gov/genome/?term="

## Search Each Organism Name on NCBI and download genome if it appears in records

In [71]:
#loop through all organisms listed in MiBig
for ii in np.arange(0,5):# set to np.arange(len(mibig)) for full download
    print ii
    # create url to parse on NCBI website
    full_url = urlbase+mibig['Organism'].iloc[ii].replace(' ','%2B')
    page = urllib2.urlopen(full_url)
    soup = BeautifulSoup(page)
    
    #extract title of search with beautiful soup
    title = soup.html.head.title
    title = str(title).rstrip('  - Genome - NCBI</title>').lstrip('<title>').replace(' ','_')
    links = soup.find_all('a')
    for link in links:
        tmp = link.get('href')
        #find link for fna download
        if 'fna.gz' in str(tmp):
            print tmp
            # download fna.gz file
            urllib.urlretrieve(tmp,title+'.fna.gz')
            f = gzip.open(title+'.fna.gz', 'rb')
            file_content = f.read()
            f.close()
            
            # open fna.gz file and write to fasta
            f = open('./RawFasta/'+title+'.fa','w')
            for line in file_content:
                f.write(line)
            f.close()
            # pause for a second to avoid abusing site
            time.sleep(0.5)

0
ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/204/155/GCF_000204155.1_ASM20415v1/GCF_000204155.1_ASM20415v1_genomic.fna.gz
1
ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/819/545/GCF_000819545.1_ASM81954v1/GCF_000819545.1_ASM81954v1_genomic.fna.gz
2
3
ftp://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/165/485/GCF_000165485.1_ASM16548v1/GCF_000165485.1_ASM16548v1_genomic.fna.gz
4
